In [ ]:
# Detect Nearly Vertical or Horizontal Lines (Any Color)

# Step 1: Install OpenCV (if not already installed)
# (Uncomment the next line if needed)
# !pip install opencv-python-headless

# Step 2: Import Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("hello")

# Step 3: Load Image from Local Path
# Set your local image path here
image_path = '../data/green_line.jpg'  # <-- Change this to your image filename

# Step 4: Load and Preprocess Image
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Step 5: Edge Detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Step 6: Detect Lines with Probabilistic Hough Transform
height, width = img.shape[:2]
min_length = int(0.8 * min(height, width))

lines_p = cv2.HoughLinesP(
    edges,
    1,
    np.pi / 180,
    threshold=50,
    minLineLength=min_length // 2,
    maxLineGap=10
)

# Step 7: Detect Lines with Standard Hough Transform
lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)

# Step 8: Draw Detected Lines
output_img_p = img.copy()
output_img = img.copy()

# Draw Probabilistic Hough Lines
if lines_p is not None:
    for line in lines_p:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        angle = abs(angle)
        if (angle < 10) or (abs(angle - 90) < 10):
            cv2.line(output_img_p, (x1, y1), (x2, y2), (255, 0, 0), 2)

# Draw Standard Hough Lines
if lines is not None:
    for rho_theta in lines:
        rho, theta = rho_theta[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))

        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        angle = abs(angle)
        if (angle < 10) or (abs(angle - 90) < 10):
            cv2.line(output_img, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Step 9: Show Results
def show_image(title, image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.title(title)
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.show()

show_image("Original Image", img)
show_image("Edges (Canny Output)", cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR))
show_image("Detected Lines (HoughLinesP)", output_img_p)
show_image("Detected Lines (HoughLines)", output_img)

# Optional: Save outputs
cv2.imwrite('detected_lines_output_p.png', output_img_p)
cv2.imwrite('detected_lines_output_standard.png', output_img)
print("Output images saved as detected_lines_output_p.png and detected_lines_output_standard.png")
